In [1]:
from google.cloud import storage
import pandas as pd
import io

bucket_name="project1text2sql"
file_name="対象案件情報_CS.xlsm"
project_id="qwiklabs-asl-02-ea8bd4c3a2f7"

# Google Cloud Storage クライアントを初期化
storage_client = storage.Client(project=project_id)

# バケットとファイルオブジェクトを取得
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_name)

# ファイルの内容をメモリにダウンロード
content = blob.download_as_bytes()

# BytesIOでファイルのように扱う
with io.BytesIO(content) as in_memory_file:
    # pandasでExcelファイルを読み込み
    df = pd.read_excel(in_memory_file, engine="openpyxl")  # engineは必要に応じて変更

# DataFrameの内容を表示
print(df)

   Unnamed: 0 Unnamed: 1 Unnamed: 2  \
0        受付番号         店番        支店名   
1        1155        124         深川   
2        1156        056        松飛台   
3        1158        246        大多喜   
4        1163        213         君津   
5        1165        046         行徳   
6        1171        046         行徳   
7        1178        201         八幡   
8        1179        001         中央   
9        1200        026       千葉駅前   
10       1201        215        袖ケ浦   
11       1202        085         誉田   
12       1205        006         蘇我   
13       1207        224         館山   
14       1212        126   流山おおたかの森   
15       1215        207         富津   
16       1218        201         八幡   
17       1225        041        勝田台   
18       1228        112       新八千代   
19       1230        229         鴨川   
20       1234        289         志津   
21       1236        003         稲毛   
22       1237        005         長洲   
23       1239        072        新浦安   
24       1243        020 

In [2]:
type(df)

pandas.core.frame.DataFrame

In [3]:
rec_number_df=df["Unnamed: 0"].iloc[1:]

In [4]:
rec_number_droplist=["1228","1230"]
rec_number_list=list(rec_number_df)
for number in rec_number_droplist:
    rec_number_list.remove(number)
rec_number_list

['1155',
 '1156',
 '1158',
 '1163',
 '1165',
 '1171',
 '1178',
 '1179',
 '1200',
 '1201',
 '1202',
 '1205',
 '1207',
 '1212',
 '1215',
 '1218',
 '1225',
 '1234',
 '1236',
 '1237',
 '1239',
 '1243']

In [5]:
from google.cloud import storage

storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_name)
data = blob.download_as_string()
COLUMNS=["受付番号","店番","支店名","依頼","SQL"]

In [6]:
req_list=pd.DataFrame(columns=COLUMNS)
for SHEET_NAME in rec_number_list:
    try:
        df = pd.read_excel(io.BytesIO(data),SHEET_NAME)
        A2=df["受付番号"].iloc[0]
        B2=df["店番"].iloc[0]
        C2=df["支店名"].iloc[0]
        D2=df["依頼"].iloc[0]
        E5=df["Unnamed: 4"].iloc[3]
        req_info = [A2,B2,C2,D2,E5]
        df_req_info=pd.DataFrame([req_info],columns=COLUMNS)
        req_list=pd.concat([req_list,df_req_info],ignore_index=True)
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        exit(1)  

req_list['依頼番号']=req_list['受付番号'].astype(int)
req_list['3桁店番']=req_list['店番'].astype(int).astype(str).str.zfill(3)
req_list.drop(columns={'受付番号', '店番'},inplace=True)
req_list = req_list.reindex(columns=['依頼番号', '3桁店番', '支店名', '依頼','SQL'])

/var/tmp/ipykernel_1530276/2924236475.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  req_list=pd.concat([req_list,df_req_info],ignore_index=True)


In [7]:
req_list

,依頼番号,3桁店番,支店名,依頼,SQL
0,1155,124,深川,【受付番号】\n1155\n【依頼内容】\n一般貸出における変動金利債権の抽出\n\n【抽出...,PROC SQL;\n CREATE TABLE WORK.'124_データリスト'n A...
1,1156,056,松飛台,\n松飛台支店の変動金利貸出の債権ごとのリストをいただけますでしょうか。,PROC SQL;\n CREATE TABLE WORK.'変動金利債権_事業資金_7'...
2,1158,246,大多喜,\n条件：保証協会利用先。債権毎、各債権の当初実行額と7月末残高、融資コード（できれば制度名...,PROC SQL;\n CREATE TABLE WORK.'QUERY_FOR_融資ロー...
3,1163,213,君津,\n【依頼内容】\nブルーベリーを栽培する法人から、ブルーベリーを用いたジャムをOEMで製造...,PROC SQL;\nCREATE TABLE WORK.'融資ジャムリスト' AS\nSE...
4,1165,046,行徳,\n【依頼内容】\n一般、融資先数増加推進の為、直近1年分の与信解消先リスト作成頂きたいです...,\nPROC SQL;\nCREATE TABLE WORK.'最終結果'n AS\nSEL...
5,1171,046,行徳,\n【依頼内容】\n融資新規推進のため、支店で新規先となる法人のリストアップをお願いいたしま...,PROC SQL;\n CREATE TABLE WORK.'QUERY_FOR_新規融資...
6,1178,201,八幡,\n【依頼内容】\nATM手数料が毎月500円以上かかっている先\n【使途】\nクロスユース...,\n\nレスポンス\nPROC SQL;\nCREATE TABLE WORK.'正規化'n...
7,1179,001,中央,\nコンサルティングプラザで申込済み（118千葉）の方で、\n医療保険契約先リスト（001中...,"PROC SQL;\nCREATE TABLE WORK.'QUERY_FOR_118,AA..."
8,1200,026,千葉駅前,\n【依頼内容】\nちばぎんアプリ推進のため顧客一覧を作成頂きたいです。\n\n【抽出条件】...,PROC SQL;\n CREATE TABLE WORK.list AS\n SELE...
9,1201,215,袖ケ浦,\n本日発刊通達 アプリＤＭリスト袖ヶ浦支店分で、給振がある先のリストアップ＆更に何処からの...,PROC SQL;\nCREATE TABLE WORK.リスト AS\nSELECT DI...


In [8]:
import csv
import os

DATA_DIR = "data"
FILE_NAME = "request_list.csv"

file_path = os.path.join(DATA_DIR, FILE_NAME)

req_list.to_csv(file_path, index=False, mode='w')